## Explore `sdf` output
`sdf` generates a large amount of information during fitting. Most of this is saved in a database that isn't visible on the web, and also in `pickle` files that can be found for each model under the "..." link.

A simpler output is the `json` files under the "model" link, there is less detail here, but they are sufficient for plotting.

To just explore the output you can probably avoid installing the `sdf` package.

In [1]:
import requests
import pickle

### json output
To explore json output we don't need any special packages. Either download directly with `requests`, or open with the `json` module.

In [2]:
r = requests.get('http://drgmk.com/sdb/seds/masters/'
                 'sdb-v2-132436.10-513016.1/public/sdb-v2-132436.10-513016.1-mnest/phoenix_m+modbb_disk_r_.json')

In [3]:
d = r.json()

In [4]:
for k in d.keys():
    print(k, type(d[k]))

id <class 'str'>
write_time <class 'float'>
model_comps <class 'list'>
main_results <class 'list'>
parameters <class 'list'>
best_params <class 'list'>
best_params_1sig <class 'list'>
chisq <class 'float'>
phot_band <class 'list'>
phot_wavelength <class 'list'>
phot_fnujy <class 'list'>
phot_e_fnujy <class 'list'>
phot_upperlim <class 'list'>
phot_ignore <class 'list'>
model_comp_fnujy <class 'list'>
model_comp_fnujy_1sig <class 'list'>
model_total_fnujy <class 'list'>
model_total_fnujy_1sig <class 'list'>
spectra <class 'list'>
model_spectra <class 'list'>
star_spec <class 'dict'>
disk_spec <class 'dict'>


The information contained in the json is largely related to the observational data, e.g. photometry and models in the observed bands.

There are also spectra for each model component.

### pickle output
To explore the pickle data we need the `pickle` package. There is a tonne of information saved here, including fluxes for the models in all bands known to `sdf`.

In [5]:
s = requests.get('http://drgmk.com/sdb/seds/masters/'
                 'sdb-v2-132436.10-513016.1/public/sdb-v2-132436.10-513016.1-mnest/phoenix_m+modbb_disk_r_.pkl')

In [6]:
r = pickle.loads(s.content)

 Config
  found files:
['/Users/grant/Library/CloudStorage/OneDrive-UniversityofWarwick/astro/projects/sdf/sdf/sdf/sdf.conf']
  no models in sdf/data/models/
  may need to create ~/.sdf.conf


In [7]:
# print the model component fluxes for the MIRI bands
print(f'filter: {r.model_comps}, total')
for i,f in enumerate(r.all_filters):
    if 'NIRCAM' in f:
        print(f, r.all_comp_phot[:,i], r.all_phot[i])

filter: ('phoenix_m', 'modbb_disk_r'), total
NIRCAM.F150W [2.13198342e+00 2.34871722e-08] 2.131984512453575
NIRCAM.F405N [0.37867233 0.00060907] 0.37977122579817574
NIRCAM.F323N [5.97448948e-01 1.58234883e-04] 0.5979389866530374
NIRCAM.F210M [1.27483207e+00 2.53378948e-06] 1.2749226875554802
NIRCAM.F150W2 [2.08534787e+00 9.25090669e-07] 2.085361309952109
NIRCAM.F212N [1.25275230e+00 2.81024833e-06] 1.2529138025740396
NIRCAM.F444W [0.34194737 0.00090665] 0.34332980317784306
NIRCAM.F322W2 [6.35532763e-01 1.96672480e-04] 0.6359075787536583
NIRCAM.F356W [5.03588441e-01 3.12083594e-04] 0.5043522468969883
NIRCAM.F410M [0.38742794 0.0006344 ] 0.38861480314580754
NIRCAM.F187N [1.44771356e+00 5.31897042e-07] 1.4477258729485456
NIRCAM.F430M [0.35474033 0.00079253] 0.35602958177886135
NIRCAM.F277W [7.94793223e-01 5.10092785e-05] 0.7950676641182637
NIRCAM.F090W [3.83644138e+00 2.20437153e-10] 3.8364413751864346
NIRCAM.F070W [4.72662259e+00 4.01142223e-10] 4.726622590361215
NIRCAM.F466N [0.29056508